# Section 2: Feature Store（特徴量ストア）

## 🎯 このセクションの目標
- Feature Storeの概念と利点を理解する
- 顧客・注文データから特徴量を計算する
- 計算した特徴量をFeature Storeに登録する
- バージョン管理（v1 → v2）を体験する

## 📋 チャーンの定義

> **2024年前半（1-6月）に注文があったが、2024年後半（7-12月）に注文がなかった顧客**

| 期間 | 用途 |
|------|------|
| 2024年前半（1-6月） | 特徴量計算（この期間の行動を分析） |
| 2024年後半（7-12月） | チャーン判定（この期間に注文なし＝離反） |

## 💡 Feature Storeとは？

Feature Storeは、機械学習の特徴量を一元管理するためのシステムです。

**主な利点:**
- 🔄 **再利用性**: 一度定義した特徴量を複数のモデルで共有
- 📊 **一貫性**: 学習と推論で同じ特徴量定義を使用
- 🔄 **バージョン管理**: 特徴量の変更履歴を追跡

---
## 2.1 環境設定

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *

# Feature Store関連
from snowflake.ml.feature_store import (
    FeatureStore,
    FeatureView,
    Entity,
    CreationMode
)

session = get_active_session()
session.use_database("MLOPS_HOL_DB")
session.use_warehouse("MLOPS_HOL_SQL_WH")

print("✅ ライブラリのインポート完了")

---
## 2.2 データの確認

まず、生データ（顧客・注文）を確認します。

In [ ]:
# 顧客データ
customers = session.table("PREP_DATA.CUSTOMERS")
print(f"顧客数: {customers.count():,}")
customers.show(5)

In [ ]:
# 注文データ
orders = session.table("PREP_DATA.ORDERS")
print(f"注文数: {orders.count():,}")
orders.show(5)

In [ ]:
# チャーンラベル（01_DATA_EXPLORATION で作成）
churn_labels = session.table("PREP_DATA.CHURN_LABELS")
print(f"チャーン率: {churn_labels.agg(F.avg('IS_CHURNED')).collect()[0][0]*100:.1f}%")
churn_labels.group_by("IS_CHURNED").count().show()

print("\n💡 このラベルは 01_DATA_EXPLORATION で作成したものです")

---
## 2.3 特徴量の計算

顧客・注文データから **RFM特徴量** を計算します。

### RFM分析とは？

| 指標 | 説明 | チャーンとの関係 |
|------|------|-----------------|
| **R**ecency | 最終注文からの日数 | 長い → チャーンしやすい |
| **F**requency | 注文回数 | 少ない → チャーンしやすい |
| **M**onetary | 注文金額 | 低い → チャーンしやすい |

In [ ]:
# Use data up to observation date (June 30, 2024)
from datetime import date
OBSERVATION_DATE = date(2024, 6, 30)

# Filter orders up to June 30, 2024
orders_filtered = orders.filter(F.col("ORDER_DATE") <= F.lit(OBSERVATION_DATE))

# Aggregate orders by customer
customer_order_stats = orders_filtered.group_by("CUSTOMER_ID").agg(
    # Recency: days since last order
    F.datediff("day", F.max("ORDER_DATE"), F.lit(OBSERVATION_DATE)).alias("DAYS_SINCE_LAST_ORDER"),
    
    # Frequency: order count
    F.count("*").alias("TOTAL_ORDER_COUNT"),
    
    # Monetary: amount
    F.sum("ORDER_AMOUNT").alias("TOTAL_ORDER_AMOUNT"),
    F.avg("ORDER_AMOUNT").alias("AVG_ORDER_AMOUNT")
)

print("=== 基本RFM特徴量（2024/6/30時点）===")
customer_order_stats.show(5)

In [ ]:
# Aggregate orders for H1 2024 (Jan-Jun)
orders_2024_h1 = orders.filter(
    (F.col("ORDER_DATE") >= "2024-01-01") & 
    (F.col("ORDER_DATE") <= "2024-06-30")
)

order_stats_2024 = orders_2024_h1.group_by("CUSTOMER_ID").agg(
    F.count("*").alias("ORDER_COUNT_2024_H1"),
    F.sum("ORDER_AMOUNT").alias("TOTAL_AMOUNT_2024_H1")
)

print("=== 2024年前半（H1）の注文集計 ===")
order_stats_2024.show(5)

In [ ]:
# Calculate return rate (data up to June 30, 2024)
return_stats = orders_filtered.group_by("CUSTOMER_ID").agg(
    (F.sum(F.when(F.col("STATUS") == "RETURNED", 1).otherwise(0)) / F.count("*")).alias("RETURN_RATE")
)

print("=== 返品率 ===")
return_stats.show(5)

In [ ]:
# Join all features
feature_df = customer_order_stats \
    .join(order_stats_2024, on="CUSTOMER_ID", how="left") \
    .join(return_stats, on="CUSTOMER_ID", how="left") \
    .join(customers.select("CUSTOMER_ID", "SEGMENT"), on="CUSTOMER_ID", how="left") \
    .fillna(0)

# Organize column order
feature_df = feature_df.select(
    "CUSTOMER_ID",
    "DAYS_SINCE_LAST_ORDER",
    "TOTAL_ORDER_COUNT",
    "ORDER_COUNT_2024_H1",
    "TOTAL_ORDER_AMOUNT",
    "AVG_ORDER_AMOUNT",
    "TOTAL_AMOUNT_2024_H1",
    "RETURN_RATE",
    "SEGMENT"
)

print(f"特徴量レコード数: {feature_df.count():,}")
print(f"カラム: {feature_df.columns}")
feature_df.show(5)

---
## 2.4 Feature Storeの作成

In [ ]:
# Feature Storeの作成
fs = FeatureStore(
    session=session,
    database="MLOPS_HOL_DB",
    name="FEATURE_STORE",
    default_warehouse="MLOPS_HOL_SQL_WH",
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST
)

print("✅ Feature Store 作成完了")

In [ ]:
# Cleanup existing objects (for re-execution)
try:
    session.sql('DROP VIEW IF EXISTS FEATURE_STORE."CUSTOMER_FEATURES$v1"').collect()
    session.sql('DROP VIEW IF EXISTS FEATURE_STORE."CUSTOMER_FEATURES$v2"').collect()
    session.sql('DROP TABLE IF EXISTS FEATURE_STORE.TRAINING_DATASET_V1').collect()
    print("Cleaned up existing objects")
except Exception as e:
    print(f"Cleanup: {e}")

---
## 2.5 Entityの定義

**Entity（エンティティ）** は、特徴量が紐づく主キーを定義します。

In [ ]:
# Define customer entity
customer_entity = Entity(
    name="CUSTOMER",
    join_keys=["CUSTOMER_ID"],
    desc="Entity for customer identification"
)

fs.register_entity(customer_entity)
print("✅ Entity 'CUSTOMER' を登録しました")

---
## 2.6 FeatureView v1の登録

まず基本的なRFM特徴量（5個）でv1を作成します。

In [ ]:
# v1: Basic RFM features (5)
feature_df_v1 = feature_df.select(
    "CUSTOMER_ID",
    "DAYS_SINCE_LAST_ORDER",
    "TOTAL_ORDER_COUNT",
    "TOTAL_ORDER_AMOUNT",
    "AVG_ORDER_AMOUNT",
    "ORDER_COUNT_2024_H1"
)

# FeatureView v1 の作成・登録
customer_fv_v1 = FeatureView(
    name="CUSTOMER_FEATURES",
    entities=[customer_entity],
    feature_df=feature_df_v1,
    desc="Customer features v1 - Basic RFM features"
)

customer_fv_v1 = fs.register_feature_view(
    feature_view=customer_fv_v1,
    version="v1"
)

print("✅ FeatureView 'CUSTOMER_FEATURES' v1 を登録しました")
print(f"   特徴量: {feature_df_v1.columns[1:]}")

---
## 2.7 バージョン管理の体験（v1 → v2）

特徴量を追加した **v2** を作成します。

### v2で追加する特徴量

| 特徴量 | 説明 |
|--------|------|
| TOTAL_AMOUNT_2024_H1 | 2024年前半の総注文金額 |
| RETURN_RATE | 返品率 |

> 💡 SEGMENTはカテゴリカル変数のため、今回のハンズオンでは省略します。

In [ ]:
# v2: Extended features (7)
feature_df_v2 = feature_df.select(
    "CUSTOMER_ID",
    "DAYS_SINCE_LAST_ORDER",
    "TOTAL_ORDER_COUNT",
    "TOTAL_ORDER_AMOUNT",
    "AVG_ORDER_AMOUNT",
    "ORDER_COUNT_2024_H1",
    # Added in v2
    "TOTAL_AMOUNT_2024_H1",
    "RETURN_RATE"
)

# FeatureView v2 の作成・登録
customer_fv_v2 = FeatureView(
    name="CUSTOMER_FEATURES",
    entities=[customer_entity],
    feature_df=feature_df_v2,
    desc="Customer features v2 - Added return rate, 2024 amount"
)

customer_fv_v2 = fs.register_feature_view(
    feature_view=customer_fv_v2,
    version="v2"
)

print("✅ FeatureView 'CUSTOMER_FEATURES' v2 を登録しました")
print(f"   特徴量: {feature_df_v2.columns[1:]}")

In [ ]:
# バージョン一覧確認
print("=== FeatureView バージョン一覧 ===")
for fv in fs.list_feature_views().collect():
    if fv['NAME'] == 'CUSTOMER_FEATURES':
        print(f"  - {fv['NAME']} v{fv['VERSION']}: {fv['DESC']}")

---
## 2.8 学習データセットの作成

Feature Storeから特徴量を取得し、チャーンラベルと結合します。

In [ ]:
# Get churn labels
labels_df = session.table("PREP_DATA.CHURN_LABELS")

# Get v2 features
fv = fs.get_feature_view("CUSTOMER_FEATURES", "v2")

# Create training dataset with generate_training_set
training_df = fs.generate_training_set(
    spine_df=labels_df,
    features=[fv],
    spine_label_cols=["IS_CHURNED"]
)

print(f"Training dataset: {training_df.count():,} rows")
training_df.show(5)

In [ ]:
# Save training dataset to table
training_df.write.save_as_table(
    "MLOPS_HOL_DB.FEATURE_STORE.TRAINING_DATASET_V1",
    mode="overwrite"
)

print("Saved training dataset")
print(f"Table: TRAINING_DATASET_V1")
print(f"Columns: {training_df.columns}")

---
## ✅ Section 2 完了！

### 学んだこと
- 生データ（顧客・注文）から **特徴量を計算** する方法
- Feature Storeの概念と利点
- FeatureViewの作成と登録
- **バージョン管理（v1 → v2）** の実践

### 作成したもの
- Entity: `CUSTOMER`
- FeatureView: `CUSTOMER_FEATURES` (v1: 5特徴量, v2: 7特徴量)
- 学習データセット: `TRAINING_DATASET_V1`

### 特徴量一覧

| 特徴量 | v1 | v2 | 説明 |
|--------|----|----|------|
| DAYS_SINCE_LAST_ORDER | ✓ | ✓ | 最終注文からの日数 |
| TOTAL_ORDER_COUNT | ✓ | ✓ | 総注文回数 |
| TOTAL_ORDER_AMOUNT | ✓ | ✓ | 総注文金額 |
| AVG_ORDER_AMOUNT | ✓ | ✓ | 平均注文金額 |
| ORDER_COUNT_2024_H1 | ✓ | ✓ | 2024年前半の注文回数 |
| TOTAL_AMOUNT_2024_H1 | - | ✓ | 2024年前半の注文金額 |
| RETURN_RATE | - | ✓ | 返品率 |

### 次のステップ
**Section 3: Model Training** では、Feature Storeから取得した特徴量でモデルを学習します。

➡️ `03_MODEL_TRAINING` Notebookに進んでください